## <center>Деревья решений: классификация и регрессия</center>

#### <center> Автор материала: Алексей Борисихин

### Описание задачи

Реализована модель дерева решений для задач классификации и регрессии. Для построения дерева, по аналогии с реализацией деревьев в scikit-learn, используется модифицированный [алгоритм CART](https://en.wikipedia.org/wiki/Predictive_analytics#Classification_and_regression_trees_.28CART.29).

### Описание математической основы

#### Построение деревьев решений

Для начала рассмотрим задачу классификации для объектов только с категориальными признаками и механизм построения решающих деревьев, основанный на концепции информационной энтропии (алгоритм [ID3](https://en.wikipedia.org/wiki/ID3_algorithm)).

Пусть у нас имеется выборка $X$ объектов и вектор ответов $y$.

Энтропия, интуитивно, это мера непоределенности в системе. Итак, энтропия Шеннона для системы с $K$ возможными состояниями определяется как:

$$
S = - \sum_{k=1}^{K} p_k \log_2 p_k
$$

где $p_k$ - вероятность нахождения системы в $k$-ом состоянии (фактически, доля объектов класса $k$ в выборке):

$$
p_k = \dfrac {N_k}{N} = \dfrac {\sum_i [y_i = k]}{\mid y \mid}
$$

Это выражение можно трактовать как оценку среднего количества информации, необходимого для определения класса примера из выборки. Максимум энтропии достигается, когда каждое состояние системы равновероятно.

Если выполнить разбиение выборки по категориальному признаку $x_j$, то ту же оценку среднего количества информации можно записать как:

$$
S_j = \sum_{i=1}^{J} \frac {N_i} {N} S_i
$$

где $J$ - число групп после разбиения, $S_i$ - энтропия в каждой подвыборке.

В качестве критерия выбора признака разбиения можно ввести понятие прироста информации (information gain, IG):

$$
IG_j = S - S_j = S - \sum_{i=1}^{J} \frac {N_i} {N} S_i
$$

Тогда выбор индекса лучшего признака для разбиения запишется через следующее выражение:

$$
j^{*} = argmax_j \left( IG_j \right) = argmax_j \left( S - \sum_{i=1}^{J} \frac {N_i} {N} S_i \right)
$$

В основе алгоритма построения деревьев решений ID3 лежит принцип жадной максимизации критерия выбора признака. На каждом шаге разбиения выбирается тот признак, при разделении по которому прирост информации оказывается наибольшим. 

Разбиение выборки (построение дерева) продолжается рекурсивно до выполнения критерия останова. В базовом случае этим критерием является достижение энтропии в узле дерева значения $0$, что означает нахождение в узле объектов одного класса, или невозможность уменьшить энтропию в узле никаким разбиением.

#### Использование вещественных признаков

В алгоритмах CART и [C4.5](https://en.wikipedia.org/wiki/C4.5_algorithm) добавляется возможность работы с вещественными признаками по следующей схеме. Рассмотрим вещественный признак с индексом $j$. Определим пороги разбиения выборки по этому признаку как упорядоченные уникальные значения этого признака на выборке:

$$
x_j \in \left \{ t_1, t_2, \dots, t_k \right \}_{}
$$

Тогда, выбрав параметр $\theta$, выборку можно разделить по предикату $x_j \le t_m$:

$$
\theta = \left( j, t_m \right )
$$

$$
X_L(\theta) = (X, y) \mid x_j \le t_m
$$

$$
X_R(\theta) = (X, y) \mid x_j > t_m
$$

И выбор лучшего признака и его порога разбиения будет выглядеть как:

$$
\theta^{*} = argmax_\theta (IG_\theta) = argmax_\theta \left( S - \dfrac {N_L} {N} S_L - \dfrac {N_R} {N} S_R \right)
$$

#### Уточнение механизма обработки категориальных признаков

В некоторых подходах бинарной классификации обработку категориальных признаков без предобработки их значений (LabelEncoder и т.д.)  предлагается проводить по следующей схеме. Первым шагом выполняется упорядочивание значений категориального признака $x_j$ на основе того, какая доля объектов обучающей выборки с данным значением признака придалежит классу "+1":

$$
x_j \in \{ q_1, q_2, \dots, q_k \}
$$

$$
\dfrac {1}{N_{q_1}} \sum_{x \in X(x_j = q_1)} [ y = +1 ] \le \dots \le \dfrac {1}{N_{q_k}} \sum_{x \in X(x_j = q_k)} [ y = +1 ]
$$

Этим упорядочиванием мы вводим естественный порядок значений категориального признака и можем заменить значения $q_i$ на число $i$. Теперь разбиения вершины дерева можно выполнять по предикату $x_j \le i$.

В рамках работы предполагается поддержка многоклассовой задачи классификации, и данный подход оказывается неприменим. Категориальные признаки поддерживаться не будут, необходима их предобработка до получения бинарных признаков (применение техник, подобных OneHotEncoder).

#### Использование различных критериев разбиения

Итак, мы задали функционал для построения деревьев решений на выборках с категориальными и вещественными признаками, который нужно максимизировать на каждом этапе разбиения выборки:

$$
Q \left( X, j, t \right) = F(X) - \dfrac {\mid X_L \mid} {\mid X \mid} F(X_L) - \dfrac {\mid X_R \mid} {\mid X \mid} F(X_R)
$$

где критерий качества разбиения $F(X)$ нами был определен как энтропия Шеннона S(X):

$$
F(X) = S(X) = - \sum_{k=1}^{K} p_k log_2 p_k
$$

В задачах классификации часто используется и другой критерий - [неопределенность Джини](https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity). Он определяется следующим образом:

$$
F(X) = 1 - \sum_{k=1}^{K} p_k^2
$$

Неопределенность Джини можно трактовать как вероятность ошибки классификации объекта, если классифицировать его случайно в соответствии с распределением классов в узле. Является мерой однородности от $0$ (однородной) до $1$ (гетерогенной).

#### Задача регрессии

При решении задачи регрессии идея построения дерева остается той же, но используются другие критерии качества разбиения. Они способствуют выбору таких параметров разбиения, при которых значения признаков в подвыборках примерно равны.

Дисперсия ответов вокруг среднего:

$$
F(X) = D(X) = \dfrac {1}{N} \sum_{i=1}^{N} \left( y_i - \dfrac {1}{N} \sum_{j=1}^{N} y_j \right) ^ 2
$$

Среднее отклонение ответов от медианы:

$$
F(X) = \dfrac {1}{N} \sum_{i=1}^{N} \mid y_i - med(y) \mid
$$

#### Ответы в листах

В качестве ответов в листьях дерева будут приниматься: наиболее многочисленный класс у объектов в листе - для задачи классификации, среднее значение ответов по объектам листа - для задачи регрессии:

$$
prediction_C(X) = argmax_k (p_k) = argmax_k \left( \dfrac {\sum_{i=1}^{N} [y_i = k]}{\mid y \mid} \right)
$$

$$
prediction_R(X) = \dfrac {1}{N} \sum_{i=1}^{N} y_i
$$

#### Работа с пропусками в данных

Одним из преимуществ деревьев решений является их способность работать с пропущенными данными. Рассмотрим схемы, применяемые в алгоритмах построения деревьев решений.

Пусть на стадии обучения во время поиска оптимального разбиения в вершине встречается признак с пропущенными значениями. Обозначим подвыборку объектов с пропущенными значениями признака $x_j$ как $V_j$. Тогда при расчете функционала мы можем просто проигнорировать эти объекты, внося поправку на потерю информации от этого:

$$
Q'(X, j, t) \approx \dfrac {\mid X \backslash V_j \mid}{\mid X \mid} Q(X \backslash V_j, j ,t)
$$

Если в качестве оптимального разбиения будет выбран предикат с рассмотренным нами признаком, то перед нами встает вопрос, каким образом распределить по дочерним вершинам объекты с пропущенными значениями.

---

В алгоритме C4.5 предлагается включать объекты из $V_j$ как в левый, так и в правый дочерний узел, но с соответствующими весами:

$$
w_L = \dfrac {\mid X_L \mid}{\mid X \mid}; \: w_R = \dfrac {\mid X_R \mid}{\mid X \mid}
$$

Эти веса в дальнейших операциях будут использоваться как коэффициенты перед индикаторами $[y_i = k]$ в задаче классификации.

На стадии прогнозирования, если объект попал в вершину, предикат которой не может быть вычислен из-за пропусков в данных, то прогноз для объекта вычисляется как сумма прогнозов обоих поддеревьев этой вершины с весами, пропорциональными количеству объектов в поддеревьях:

$$
prediction(x) = \dfrac {\mid X_L \mid}{\mid X \mid} prediction_L(x) + \dfrac {\mid X_R \mid}{\mid X \mid} prediction_R(x)
$$

---

Алгоритм CART предлагает использовать метод суррогатных разбиений. Этим методом будем пользоваться и мы. Основной идеей метода является поиск такого альтернативного (суррогатного) разбиения, которое максимально похоже на выбранное нами оптимальное. Мера похожести вычисляется следующим образом:

$$
similarity = \dfrac {\mid X_L^* \cap X_L^{'} \mid}{\mid X \mid} + \dfrac {\mid X_R^* \cap X_R^{'} \mid}{\mid X \mid}
$$

где $\left( X_L^*, X_R^* \right)$ - оптимальное разбиение, $\left( X_L^{'}, X_R^{'} \right)$ - суррогатное разбиение.

В случае, когда найденное значение $similarity < 0.5$, достаточно в суррогатном разбиении поменять местами левую и правую ветвь и заменить значение на $(1 - similarity)$.

Также вводится разбиение "по большинству" (пример отправляется в ветвь, содержащую большую часть выборки). Его мера похожести определяется как 

$$
similarity = max \left( \dfrac {\mid X_L^* \mid}{\mid X \mid}, \dfrac {\mid X_R^* \mid}{\mid X \mid} \right)
$$

и она является порогом, по которому отбрасываются суррогатные разбиения с меньшим значением $similarity$.

Все найденные суррогатные разбиения ранжируются и сохраняются для каждой вершины дерева. Если встречаются примеры выборки с пропущенным значением основного предиката разбиения, то поочередно проверяются суррогатные предикаты, пока не будет найден такой с непропущенным значением признака. Если этого не произошло, применяется разбиение "по большинству".

### Файлы проекта

- <b>dt_class.py</b>

    Объявление и реализация класса модели.<br><br>
    
- <b>estimator_test.ipynb</b>

    Notebook, содержащий тестирование класса модели.

### Тестирование модели

Для тестирования полученной модели и сравнения ее с деревьями решений из библиотеки sklearn были использованы средства этой библиотеки для генерации датасетов: [make_classification](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html) и [make_regression](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_regression.html). Тестирование для задач классификации и регрессии было проведено в три этапа: 
- с помощью кросс-валидации были получены показатели качества (ROC-AUC и MSE) для разных величин максимальной глубины дерева;
- также с помощью кросс-валидации проводился замер качества полученной модели на датасетах с разной долей пропущенных значений (деревья sklearn не работают с ними);
- сравнивалось качество суррогатных и константных разбиений на тестовых датасетах с разной долей пропущенных значений (обучение было проведено на датасете без пропусков в данных).

Сразу хочется отметить разницу в скорости работы алгоритмов. Классы деревьев sklearn превосходят полученный алгоритм почти на два порядка.

В задаче классификации был сгенерирован датасет размером 1000 примеров со следующими характеристиками: число информативных признаков - 5, число избыточных признаков (линейные комбинации информативных) - 3, число случайно сгенерированных шумовых признаков - 2, число кластеров для каждого из классов - 5.

Созданный класс сравнивался с классом [sklearn.tree.DecisionTreeClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html). Оба объекта были созданы с параметрами по-умолчанию и критерием разбиения Джини. На датасете без пропусков выполнялся поиск оптимального значения глубины дерева на интервале \[2, 15\] с 5-кратной кросс-валидацией для каждого значения параметра. Графики зависимости качества классификации (был выбран критерий ROC-AUC) от значения максимальной глубины дерева:
<img src='./img/comparsion_gini.png'>
Можно наблюдать более высокое качество классификации на деревьях малой глубины у класса sklearn. При росте глубины дерева значения качества классификации сближаются. У полученной модели график на тестовой подвыборке выглядит более сглаженным и с меньшим значением стандартного отклонения.

Проведем те же операции для деревьев с энтропийным критерием разбиения.
<img src='./img/comparsion_entropy.png'>
Картина осталась похожей на предыдущий случай, плюс сглаженность и меньшее значение стандартного отклонения на тестовой подвыборке стали более выраженными.

Перейдем ко второму шагу тестирования, проверим работу моделей на данных с пропусками. Т.к. деревья решений sklearn такую возмоность не поддерживают, остановимся на созданной модели. В качестве критерия разбиения будем использовать gini, а глубину ограничим значением 7. Пропуски будем генерировать случайно, во всех признаках выборки, с разными значениями доли от общего объема: от 0 до 0.5 с шагом 0.05. Для каждого значения доли проведем 5-кратную кросс-валидацию для двух версий дерева: с константным распределением пропусков (при каждом разбиении относятся к одной и той же ветви) и с использованием суррогатных разбиений.
<img src='./img/missing_classification.png'>
Суррогатные разбиения в среднем дают лучший результат, но ненамного, в рамках стандартного отклонения.

Третий шаг тестирования. Проверим работу моделей на данных с пропусками по другой схеме. Оставим значения долей пропусков и версии дерева, но теперь обучение будем производить на тренировочной части без пропусков в данных, а тестирование - на части с пропусками.
<img src='./img/test_missing_classification.png'>
В этом случае разница уже ощутима.

Перейдем к задаче регрессии. Был сгенерирован датасет из 1000 примеров с параметрами: информатвных признаков - 7, избыточных - 3, величина стандартного отклонения гауссовского шума, добавленного к данным - 0.05. Проведем те же операции, что и в задаче классификации. Критерием качества решения задачи будет служить MSE.

Для сравнения были взяты классы с параметрами по-умолчанию: [sklearn.tree.DecisionTreeRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor) с критерием MSE и два полученных дерева, с критерием разбиения variance и mad_median.

Результат поиска оптимальной глубины дерева.
<img src='./img/comparsion_regression.png'>
Картина похожа на случай классификации. Такое же преимущество у дерева sklearn с небольшим значением глубины, и также близкими значения качества на больших значениях глубины (только теперь сходство почти полное).

Повторим второй шаг тестирования. Рассмотрим полученную модель с критерием разбиения variance и максимальной глубиной дерева 8.
<img src='./img/missing_regression.png'>
Опять, как и в случае классификации, качество решения задачи в среднем больше у дерева с суррогатными разбиениями. Но на доле пропусков 0.3 и выше разница в качестве становится больше величины стандартного отклонения, что позволяет говорить об улучшении.

И, наконец, третий шаг. Обучим регрессоры на подвыборке без пропусков, а проверим на подвыборке с пропущенными значениями.
<img src='./img/test_missing_regression.png'>
В этом случае разницу в качестве можно назвать существенной.

### Используемые источники

1. Энтропия и деревья принятия решений https://habr.com/post/171759/
2. Открытый курс машинного обучения. Тема 3. Классификация, деревья решений и метод ближайших соседей https://habr.com/company/ods/blog/322534/
3. Решающие деревья (лекция ФКН ВШЭ) https://www.hse.ru/mirror/pubs/share/215285956
3. Scikit-learn Decision Trees http://scikit-learn.org/stable/modules/tree.html
4. An Introduction to Recursive Partitioning Using the RPART Routines https://cran.r-project.org/web/packages/rpart/vignettes/longintro.pdf
5. Decision Tree: Review of Techniques for Missing Values at Training, Testing and Compatibility http://uksim.info/aims2015/CD/data/8675a122.pdf